In [1]:
%matplotlib inline

In [2]:
import pydicom
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

def ReadDICOM(data):
  DICOM_path = []
  for fpath, _, fname in os.walk(data):
    for f in fname:
      DICOM_path.append(os.path.join(fpath, f))

  return DICOM_path

def SortDICOM(path):
  DICOM_slice = []
  for p in path:
    ds = pydicom.dcmread(p)
    if hasattr(ds, "ImagePositionPatient"):
      position = ds.ImagePositionPatient
      DICOM_slice.append((position[2], p))
    
  DICOM_slice.sort(key=lambda x: float(x[0])) # sorted by float(x[0])
    
  sorted_path = [s[1] for s in DICOM_slice]
  sorted_path = np.array(sorted_path)
  return sorted_path # numpy array

def VisualizeCT(index):
  Window_level = -500
  Window_width = 1500

  p = CTdata[index]
  ds = pydicom.dcmread(p)
  CTimage = ds.pixel_array
  Windowimage = np.clip(CTimage, Window_level - Window_width/2, Window_level + Window_width/2)
  Windowimage = (Windowimage - Windowimage.min()) / (Windowimage.max() - Windowimage.min())  # [0, 1]

  plt.figure(figsize=(8, 8))
  plt.imshow(Windowimage, cmap='gray')
  plt.title("Lung")  
  plt.axis('off')
  plt.show() 
  plt.close()

    

'''Main'''
data_folders = [
  "/home/lyy/chenMLNovice/data/HW1data/data1", 
  "/home/lyy/chenMLNovice/data/HW1data/data2", 
  "/home/lyy/chenMLNovice/data/HW1data/data3", 
  "/home/lyy/chenMLNovice/data/HW1data/data4"
]
i = 0
path = ReadDICOM(data_folders[i]) 
CTdata = SortDICOM(path) 
slider = widgets.IntSlider(
  value=0,  # 初始切片索引
  min=0,    # 最小切片索引
  max= int(len(CTdata)) - 1,  # 最大切片索引
  step=1,
  description=f'Data {i+1} Slice'
)
ui = widgets.VBox([slider])
out = widgets.interactive_output(
  VisualizeCT,
  {
    'index': slider
  }
)
display(ui, out)



Output()